В рамках этого заадания мы будем создавать программу, которая получая на вход отзыв, будет предсказывать, является ли отзыв положительным или отрицательным. Делать мы будем это таким образом: мы возьмём некоторое число заранее размеченных как положительные или отрицательные отзывов, выделим те слова, которые встречаются только в положительных или только в отрицательных отзывах, и будем считать, каких слов в поступившем нам на проверку отзыве больше.

Мы будем работать по заранее определённому пайплайну:

1) Сначала нам надо скачать дату -- соберите как минимум 60 (30 положительных и 30 отрицательных) отзывов на похожие продукты (не надо мешать отзывы на отели с отзывами на ноутбуки) для составления " тонального словаря" (чем больше отзывов, тем лучше) и 10 отзывов для проверки качества. 3 балла в случае сбора путём парсинга, 1 - если найдете уже готовые данные или просто закопипастите без парсинга

2) Токенизируйте слова, приведите их к нижнему регистру и к начальной форме (1 балл за токенизацию, 1 - за начальную форму)

3) Составьте 2 множества - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с частотностями и исключить шум (к примеру, выбросить слова, встречающиеся 1-2 раза) (3 балла) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку)

4) Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1 - за коректно работающую функцию, 1 - за подсчёт accuracy)

5) Предложите как минимум 2 способа улучшить эту программу с помощью добавления к ней любых мулек - просто словами, писать улучшающий код не надо (1 балл)

Логичность и чистота кода (1 балл)

Да, тут 12 баллов - два можно потерять.

В случае, если после долгих мучений в п. 3 множества по объективным причинам не получается (покажите, что пытались) - отправляйте жабу - зачьтём полный балл

### Анна Смирнова БКЛ181


In [42]:
from html import unescape
from bs4 import BeautifulSoup
import re
import time
import html
import collections
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import requests
from fake_useragent import UserAgent
from pymystem3 import Mystem
m = Mystem()

ua = UserAgent(verify_ssl=False)
stopwords = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()
session = requests.session()

функция, которая достает хорошие, плохие и все отзывы о Сумерках с соответствующих страничек:

In [2]:
def get_review(url):
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    page = soup.find_all('span', {'itemprop': 'reviewBody'})
    return page
good = get_review('https://www.kinopoisk.ru/film/401177/reviews/ord/date/status/good/perpage/75/')
bad = get_review('https://www.kinopoisk.ru/film/401177/reviews/ord/date/status/bad/perpage/75/')
both = get_review('https://www.kinopoisk.ru/film/401177/reviews/ord/date/status/all/perpage/25/page/7/')

функция очищает текст от лишних знаков, символов новой строки, тегов и тд

In [4]:
def clean_text(raw):
    lines = []
    for line in raw:
        line = str(line)
        text = re.sub('<.*?>','',line)
        text2 = re.sub('[\n\r]',' ',text)
        lines.append(text2)
    return lines

In [5]:
clean_text(good) #тут можно посмотреть, каким вышел очищенный текст

["Единственная мысль, которая возникает во время просмотра данного фильма, исключая красивый и привлекательный антураж, выдержанные сцены в специфическом для данной картины колорите, это тема романтической любви, коими прототипами являются Ромео и Джульета (книга данного произведения даже появляется во 2 части и упоминается на уроке литературы), а также различные произведения эпохи рыцарских походов аля Дон Кихот, в которых тема любви раскрывается через пожертвование чем-то ради другого, близкого тебе человека, вплоть до самой смерти, тоже самое мы можем лицезреть и в данном фильме, герои буквально влюбляются в друг друга с одного взгляда, их чувства в самом начале настолько скованны, что кажется, как будто актеры плохо сыграли, но, как мне видится, в этом и была задумка режиссерской команды. В конечном итоге, как и во всех мелодрамах, герои не могут оставаться друг без друга, стоит им лишь разлучиться, как их сразу же кидает в морально-духовные переживания переходящие в депресуху, и н

In [6]:
clean_text(bad)

["Первое, что бросается в глаза - мрак. Беспросветный и безысходный мрак, прекрасно передающий всё наполнение фильма, который “кишит” кровопийцами, томными взглядами и всё тем же мрачным пафосом.    На самом деле, очень странно, что и режиссёр и продюсеры решили пойти на такой шаг и выпустить, по своей сути, фантастическую мелодраму максимум телевизионного уровня на большие экраны. Но заход с красавчиком Паттинсоном, известному четырнадцатилетней публике по уже отшумевшему «Кубку Огня», своё дело сделал и обеспечил неплохие кассовые сборы (и даже заработал какие-то награды!).   Скупая сюжетная линия повествует об излишне аморфной девушке Изабелле (Кристен Стюарт), которая переезжает от матери на ПМЖ к своему папаше копу. Притянутая причина смены дислокации с южной Аризоны на затерянный, северный и пасмурный штат, не так очевидна, как то, что в “новенькую” сразу же влюбляется местный секс-идол Эдвард. За бледнолицым красавчиком и его накрахмаленой семейкой ходит разная молва, но дурочке

In [26]:
clean_text(both)

["Редкое произведение может вызвать во мне столько сарказма по отношению к себе, и «Сумерки»тут переплюнули всё и вся. Истории про вампиров, будь то в кино или литературе, вызывают у меня особый интерес, так как эти сверхъестественные существа представляют собой совершенно иную форму жизни, лично для меня они имеют особую мертвую привлекательность. Как говорил Ричард Роксбург, исполнивший роль Дракулы: «И почему мысль о ком-то, крадущемся в ночи, стремящемся дотянутся до вашей шеи, так нравится некоторым людям? Эта мысль сама по себе жуткая, но она завораживает.» В XХI веке появилось 'новое слово вампирской готики', мимо которого мне, к сожалению, не удалось пройти. Книга представляет собой посредственную, невероятно наивную и типо романтическую историю, написанную в духе 'пришел сюда, сделал то, сказал это' очень бедным литературный языком. А экранизация оказалась в разы бездарнее книги. Остается только поражаться её успеху.   Во мне начинает кипеть возмущение, когда я слышу, что «Сум

функция токенизирует слова, приводит их к нижнему регистру и возвращает начальные формы

In [7]:
def tokenize(text):
    words = []
    for i in text:
        for word in nltk.word_tokenize(i.lower()):
            if word.isalpha() and word not in stopwords:
                w = morph.parse(word)[0]
                w = w.normal_form
                words.append(w)
    return words

In [8]:
good_list = tokenize(clean_text(good)) #создала новую переменную, чтобы в дальнейшем к ней обращаться
#print(good_list)

In [9]:
bad_list = tokenize(clean_text(bad))
#print(bad_list)

дальше я нашла пересечения списков, чтобы исключить слова, встречающиеся и в положит и в отриц отзывах


In [10]:
same_words = list(set(good_list)&set(bad_list))
#print(same_words)

эта функция убирает из списков слова, которые встречаются в списке выше, а еще убирает шум (я тут убрала слова, встречающиеся только 1 раз, чтобы осталось побольше данных)

In [11]:
def spisok(some_list):
    c = collections.Counter()
    new_list = []
    for i in some_list:
        if i in same_words:
            continue
        else:
            c[i] += 1
    for k,v in c.items():
        if v > 1:
            new_list.append(k)
    return(new_list)

            

In [15]:
ok = spisok(good_list) #новая переменная для обозначения списка со словами из положительных отызывов
#print(ok)

In [13]:
neok = spisok(bad_list) #и для отрицательных
#print(neok)

#### ВОТ ТУТ НОВЫЙ КУСОК КОДА ИЗ ВТОРОЙ ДОМАШКИ

In [107]:
def ne_adj(some_text):
    ne = ''
    adj = ''
    x = 0
    ana = m.analyze(some_text)
    for word in ana:
        if 'analysis' in word and len(word['analysis']) != 0:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            lex = word['text']
            #print(lex,pos)
            if ne == '':
                if pos == 'PART':
                    ne = lex
            elif pos == 'A':
                adj = lex
            else:
                if pos == 'S':
                    #print(ne, ' ', adj, ' ', lex)
                    x += 1
                    ne = ''
                    adj = ''
                elif pos != 'PART':
                    ne = ''
                else:
                    ne = lex
    return x

In [108]:
def twonouns(some_text):
    s = ''
    y = 0
    ana = m.analyze(some_text)
    for word in ana:
        if 'analysis' in word and len(word['analysis']) != 0:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            lex = word['text']
            if s == '':
                if pos == 'S':
                    s = lex
            else:
                if pos == 'S':
                    #print(s, ' ', lex)
                    y += 1
                    s = ''
                elif pos != 'S':
                    s = ''
                else:
                    s = lex
    return y

In [109]:
def adj_noun(some_text):
    adj = ''
    z = 0
    ana = m.analyze(some_text)
    for word in ana:
        if 'analysis' in word and len(word['analysis']) != 0:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            lex = word['text']
            #print(lex,pos)
            if adj == '':
                if pos == 'A':
                    adj = lex
            else:
                if pos == 'S':
                    #print(adj, ' ', lex)
                    z += 1
                    adj = ''
                elif pos != 'A':
                    adj = ''
                else:
                    adj = lex
    return z

функция, определяющая сама, положительный отзыв или отрицательный 

In [105]:
def new_detect(both):
    results = []
    cleaned = clean_text(both)
    for i in cleaned:
        good = 0
        bad = 0
        check_ne = ne_adj(i)
        check_nouns = twonouns(i)
        check_adj = adj_noun(i)
        if check_ne >5 or check_nouns > 15 or check_adj > 30: 
            bad += 1
        else:
            good += 1
        if good > bad:
            print('полож отзыв')
            results.append('good')
        else:
            print('отриц отзыв')
            results.append('bad')
    return (results)
    
    

In [44]:
def detect(both):
    res = []
    #cleaned = clean_text(both)
    #print(cleaned)
    for j in both:
        good = 0
        bad = 0
        cleaned = clean_text(j)
        tok = tokenize(cleaned)
        #print(tok)
        for i in tok:
            #print(i)
            if i in ok:
                good += 1
            if i in neok:
                bad += 1
        if good > bad:
            print(str(cleaned)[-100:] + ' положительный отзыв ))))')
            res.append('good')
        else:
            print(str(cleaned)[-100:]  + ' отрицательный отзыв ((((')
            res.append('bad')
    return(res)

на вход функции подается неочищенный текст с рандомными отзывами, функция их очищает и определяет их тональность

In [110]:
new_detect(both)

отриц отзыв
полож отзыв
отриц отзыв
полож отзыв
отриц отзыв
полож отзыв
отриц отзыв
отриц отзыв
полож отзыв
полож отзыв
полож отзыв
отриц отзыв
отриц отзыв
отриц отзыв
отриц отзыв
отриц отзыв
полож отзыв
отриц отзыв
отриц отзыв
отриц отзыв
полож отзыв
отриц отзыв
отриц отзыв
отриц отзыв
полож отзыв


['bad',
 'good',
 'bad',
 'good',
 'bad',
 'good',
 'bad',
 'bad',
 'good',
 'good',
 'good',
 'bad',
 'bad',
 'bad',
 'bad',
 'bad',
 'good',
 'bad',
 'bad',
 'bad',
 'good',
 'bad',
 'bad',
 'bad',
 'good']

In [112]:
from sklearn.metrics import accuracy_score
results = []
gold = []
for i in good:
    gold.append('good')
for i in bad:
    gold.append('bad')
results = new_detect(good) + new_detect(bad)

отриц отзыв
полож отзыв
полож отзыв
отриц отзыв
отриц отзыв
отриц отзыв
полож отзыв
полож отзыв
отриц отзыв
полож отзыв
полож отзыв
полож отзыв
полож отзыв
отриц отзыв
отриц отзыв
полож отзыв
отриц отзыв
отриц отзыв
отриц отзыв
полож отзыв
полож отзыв
полож отзыв
полож отзыв
полож отзыв
полож отзыв
полож отзыв
полож отзыв
отриц отзыв
полож отзыв
полож отзыв
отриц отзыв
отриц отзыв
полож отзыв
полож отзыв
полож отзыв
отриц отзыв
отриц отзыв
отриц отзыв
отриц отзыв
отриц отзыв
полож отзыв
отриц отзыв
отриц отзыв
отриц отзыв
отриц отзыв
отриц отзыв
отриц отзыв
полож отзыв
полож отзыв
отриц отзыв
полож отзыв
отриц отзыв
полож отзыв
отриц отзыв
отриц отзыв
полож отзыв
отриц отзыв
полож отзыв
полож отзыв
отриц отзыв
отриц отзыв
отриц отзыв
полож отзыв
полож отзыв
отриц отзыв
полож отзыв
отриц отзыв
полож отзыв
отриц отзыв
отриц отзыв
полож отзыв
отриц отзыв
отриц отзыв
отриц отзыв
отриц отзыв
отриц отзыв
отриц отзыв
отриц отзыв
полож отзыв
отриц отзыв
отриц отзыв
отриц отзыв
отриц отзыв
отри

новая accuracy упала по сравнению со старой, упс )))) 

In [113]:
print("Accuracy: %.4f" % accuracy_score(results, gold))

Accuracy: 0.5733


In [111]:
detect(both)

 P.S. но даже у этого фильма есть один маленький плюс: во время просмотра мы усмеялись до коликов.'] отрицательный отзыв ((((
утким фильмом!', '', ' ', '', '  В говне определенно что то есть, миллионы мух не могут ошибаться.'] отрицательный отзыв ((((
девушкам from Stephenie with love!', '', ' ', '', '  Теперь я готова к смерти о рук толпы фанаток!'] отрицательный отзыв ((((
отра отрицательное и это лично моё мнение.', '', ' ', '', '  Моя оценка ', '', ' ', '', '  3 из 10'] отрицательный отзыв ((((
П и ВП с завышенным ЧСВ.', '', ' ', '', ' ', 'P.S. Вампиры играют в бейсбол, а чего ещё вы ждали ?'] отрицательный отзыв ((((
о только одно из мнений, которые собираются мозаикой в истинную картину.', '', ' ', '', '  8 из 10'] положительный отзыв ))))
зые красавца, но это уже будет про книгу, а мы здесь обсуждаем фильм.', '', ' ', '', ' ', '8 из 10'] положительный отзыв ))))
го существования.", '', ' ', '', '  Финал истории более чем предсказуем.', '', ' ', '', '  1 из 10'] отрицательный отз

['bad',
 'bad',
 'bad',
 'bad',
 'bad',
 'good',
 'good',
 'bad',
 'good',
 'good',
 'bad',
 'good',
 'good',
 'bad',
 'good',
 'bad',
 'good',
 'bad',
 'good',
 'bad',
 'good',
 'bad',
 'good',
 'bad',
 'bad']

In [36]:
from sklearn.metrics import accuracy_score
results = []
gold = []
for i in good:
    gold.append('good')
for i in bad:
    gold.append('bad')
results = detect(good) + detect(bad)


'  Хоть и в некоторых моментах присутствовали нотки нереалистичности, мне понравился данный фильм.'] положительный отзыв ))))
лючить ребёнка, который верит, что любовь бывает просто так, ни за что. ', '', ' ', '', '  6 из 10'] положительный отзыв ))))
 хеппи-эндом, а вот потеряное очарование вам уже не удасться возвратить.", '', ' ', '', '  9 из 10'] положительный отзыв ))))
', '  Я же невзыскательный зритель, к тому же девушка, поэтому от меня,', '', ' ', '', '  10 из 10'] положительный отзыв ))))
атко. На грани фола. Во второй части так может уже и не прокатить. ', '', ' ', '', '  На этом всё…'] положительный отзыв ))))
мпиров! ', '', ' ', '', '  «Сумерки» надолго выбили меня из реальности.', '', ' ', '', '  10 из 10'] положительный отзыв ))))
, '  Я про то, что непервый просмотр хорошо пошел и прошел, и даже захотелось похвалить «Сумерки».'] положительный отзыв ))))
ю, что первая часть была самой лучшей и нужно было на ней остановиться.', '', ' ', '', '  10 из 10'] положительный отз

In [37]:
print("Accuracy: %.4f" % accuracy_score(results, gold))

Accuracy: 0.9867


предложения по улучшению программы: 
1) взять побольше отзывов, тк на малом количестве отзывов очень много слов, которые встречаются и в положительных, и в отрицательных отзывах, много "шума", так что при удалении всего этого, остается достаточно мало слов для определения тональность
2) было бы неплохо все в одну функцию скрепить, чтобы получилось более последовательно и понятно ))))
3) можно было сделать инпут, чтобы пользователь сам вводил любой отзыв и получал результат (я так делать не стала, но как опция)
